# House Price Predictor

In [20]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

In [21]:
# -----------------------------
# 1. Load the dataset
# -----------------------------
# Replace 'house_prices.csv' with the path to your dataset.
data = pd.read_csv('train.csv')
data
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1000 non-null   int64  
 1   MSSubClass     1000 non-null   int64  
 2   MSZoning       1000 non-null   object 
 3   LotFrontage    827 non-null    float64
 4   LotArea        1000 non-null   int64  
 5   Street         1000 non-null   object 
 6   Alley          65 non-null     object 
 7   LotShape       1000 non-null   object 
 8   LandContour    1000 non-null   object 
 9   Utilities      1000 non-null   object 
 10  LotConfig      1000 non-null   object 
 11  LandSlope      1000 non-null   object 
 12  Neighborhood   1000 non-null   object 
 13  Condition1     1000 non-null   object 
 14  Condition2     1000 non-null   object 
 15  BldgType       1000 non-null   object 
 16  HouseStyle     1000 non-null   object 
 17  OverallQual    1000 non-null   int64  
 18  OverallCo

In [22]:
# -----------------------------
# 2. Data Cleaning
# -----------------------------
# Select only numeric columns that have no missing data.
# Identify categorical columns (if they are not already of type 'category')
# Identify categorical columns (if they are not already type 'category')
categorical_cols = data.select_dtypes(include=['object', 'category']).columns

# One-hot encode these columns (original columns are dropped automatically)
data = pd.get_dummies(data, columns=categorical_cols, dtype=int,drop_first=True)

#numeric_cols = data.select_dtypes(include=[np.number]).columns
clean_numeric_cols = [col for col in data if data[col].isna().sum() == 0]
data_clean = data[clean_numeric_cols]
print(data_clean.columns)

# Ensure that the target column 'price' is present.
if 'SalePrice' not in data_clean.columns:
    raise ValueError("The target column 'price' is not present in the complete numeric data.")

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       ...
       'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth',
       'SaleType_WD', 'SaleCondition_AdjLand', 'SaleCondition_Alloca',
       'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=229)


In [23]:
# -----------------------------
# 3. Feature Selection
# -----------------------------
# Compute the correlation matrix using only the cleaned numeric data.
corr_matrix = data_clean.corr()

# Compute absolute correlations of features with the target and drop the target itself.
target_corr = corr_matrix['SalePrice'].drop('SalePrice').abs().sort_values(ascending=False)

# Select only the top 4 features with the highest correlation with 'SalesPrice'
top4_features = target_corr.head(20).index
print("Selected top 4 features:", list(top4_features))

# Define input features (X) and target variable (y).
X = data_clean[top4_features].values
y = data_clean['SalePrice'].values.reshape(-1, 1)

Selected top 4 features: ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'ExterQual_TA', 'TotRmsAbvGrd', 'FullBath', 'KitchenQual_TA', 'YearBuilt', 'YearRemodAdd', 'Foundation_PConc', 'Fireplaces', 'BsmtQual_TA', 'Neighborhood_NridgHt', 'ExterQual_Gd', 'BsmtFinType1_GLQ', 'GarageFinish_Unf', 'BsmtFinSF1']


In [24]:
# -----------------------------
# 4. Data Preprocessing
# -----------------------------
# Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features to improve training stability.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
# Convert the numpy arrays to PyTorch tensors.
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create a TensorDataset and DataLoader for batch processing.
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [26]:
# -----------------------------
# 5. Define the Neural Network Model
# -----------------------------
class HousePriceModel(nn.Module):
    def __init__(self, input_dim):
        super(HousePriceModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output layer for regression
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HousePriceModel(input_dim=X_train.shape[1]).to(device)

In [27]:
# -----------------------------
# 6. Set Up Loss Function and Optimizer
# -----------------------------
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# 7. Train the Model
# -----------------------------
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * batch_X.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [10/1000], Loss: 40836314562.5600
Epoch [20/1000], Loss: 38683634237.4400
Epoch [30/1000], Loss: 33184947650.5600
Epoch [40/1000], Loss: 24853117255.6800
Epoch [50/1000], Loss: 16167774003.2000
Epoch [60/1000], Loss: 9991072849.9200
Epoch [70/1000], Loss: 7118325504.0000
Epoch [80/1000], Loss: 6117254563.8400
Epoch [90/1000], Loss: 5634376878.0800
Epoch [100/1000], Loss: 5253980723.2000
Epoch [110/1000], Loss: 4895113697.2800
Epoch [120/1000], Loss: 4558248929.2800
Epoch [130/1000], Loss: 4246249041.9200
Epoch [140/1000], Loss: 3958298127.3600
Epoch [150/1000], Loss: 3692963031.0400
Epoch [160/1000], Loss: 3456343992.3200
Epoch [170/1000], Loss: 3244037294.0800
Epoch [180/1000], Loss: 3059558328.3200
Epoch [190/1000], Loss: 2902188282.8800
Epoch [200/1000], Loss: 2766522869.7600
Epoch [210/1000], Loss: 2651581352.9600
Epoch [220/1000], Loss: 2550278983.6800
Epoch [230/1000], Loss: 2462819069.4400
Epoch [240/1000], Loss: 2382769735.6800
Epoch [250/1000], Loss: 2308772060.1600
Epoc

In [28]:
# 8. Evaluate the Model
# -----------------------------
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor.to(device))
    test_loss = criterion(predictions, y_test_tensor.to(device)).item()
    print("Test Mean Squared Error:", test_loss)

# Optionally, to evaluate using scikit-learn's MSE:
predictions_np = predictions.cpu().numpy()
mse = mean_squared_error(y_test, predictions_np)
print("Test MSE (scikit-learn):", mse)
#Test Mean Squared Error: 935741376.0

Test Mean Squared Error: 720409856.0
Test MSE (scikit-learn): 720409856.0
